# HyDE
For a given query, HyDE retrieval pipeline contains 4 components:
1. Promptor: bulid prompt for generator based on specific task.
2. Generator: generates hypothesis documents using Large Language Model.
3. Encoder: encode hypothesis documents to HyDE vector.
4. Searcher: search nearest neighbour for the HyDE vector (dense retrieval).

### Initialize HyDE components
We use [pyserini](https://github.com/castorini/pyserini) as the search interface.

In [1]:
import json
print('1')
from pyserini.search import FaissSearcher, LuceneSearcher
print('2')
from pyserini.search.faiss import AutoQueryEncoder
print('3')

# from hyde import Promptor, OpenAIGenerator, CohereGenerator, HyDE

from hyde.promptor import Promptor
from hyde.generator import DeepSeekGenerator
from hyde.hyde import HyDE


1


/root/miniconda3/envs/hyde-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2
3


In [2]:
# KEY = '<api key>' # replace with your API key, it can be OpenAI api key or Cohere api key
# use deepseek llm

# KEY = 'sk-b0d1a733e4a44c4c91aaca082e3f9ed0'
KEY='sk-77ca6e57c7324f31a7b5ef9fc6e9b29d'#help from friend
model = 'deepseek-chat' # only one model of deepseek is supported for now

promptor = Promptor('web search')
print('1')
# generator = OpenAIGenerator('text-davinci-003', KEY)
generator = DeepSeekGenerator(model, KEY)
print('2')
# encoder = AutoQueryEncoder(encoder_dir='facebook/contriever', pooling='mean')
encoder = AutoQueryEncoder(encoder_dir='/root/autodl-tmp/HuggingFace-Model/facebook-contriever', pooling='mean')
print('3')
searcher = FaissSearcher('/root/autodl-tmp/hyde/HyDE-redisplay/contriever_msmarco_index', encoder)
print('4')
# corpus = LuceneSearcher.from_prebuilt_index('msmarco-v1-passage')
corpus = LuceneSearcher(index_dir='/root/autodl-tmp/localFile/lucene-inverted.msmarco-v1-passage.20221004.252b5e')
print('5')

1
2
3
4
5


Oct 24, 2024 8:27:46 PM org.apache.lucene.store.MemorySegmentIndexInputProvider <init>
INFO: Using MemorySegmentIndexInput with Java 21; to disable start with -Dorg.apache.lucene.store.MMapDirectory.enableMemorySegments=false


### Build a HyDE pipeline

In [3]:
hyde = HyDE(promptor, generator, encoder, searcher)

### Load example Query

In [4]:
query = 'how long does it take to remove wisdom tooth'

### Build Zeroshot Prompt

In [5]:
prompt = hyde.prompt(query)
print(prompt)

Please write a passage to answer the question.
Question: how long does it take to remove wisdom tooth
Passage:


### Generate Hypothesis Documents

In [6]:
hypothesis_documents = hyde.generate(query)
for i, doc in enumerate(hypothesis_documents):
    print(f'HyDE Generated Document: {i}')
    print(doc.strip())

HyDE Generated Document: 0
The duration to remove a wisdom tooth can vary significantly depending on several factors, including the complexity of the extraction, the position of the tooth, and the patient's overall dental health. Generally, a straightforward wisdom tooth removal, where the tooth is fully erupted and easily accessible, can take as little as 15 to 30 minutes. However, if the tooth is impacted, meaning it is trapped beneath the gum or bone, the procedure can be more involved and may require additional time, commonly ranging from 45 minutes to an hour or more. In cases of severe impaction or complications, the extraction process could extend even further. Post-operative care and recovery time can also vary, typically lasting from a few days to a couple of weeks. It is important to consult with a dental professional for a personalized assessment and to understand the specific timeline for your particular case.


### Encode HyDE vector

In [25]:
hyde_vector = hyde.encode(query, hypothesis_documents)
print(hyde_vector.shape)

(1, 768)


### Search Relevant Documents

In [26]:
hits = hyde.search(hyde_vector, k=10)
for i, hit in enumerate(hits):
    print(f'HyDE Retrieved Document: {i}')
    print(hit.docid)
    print(json.loads(corpus.doc(hit.docid).raw())['contents'])

HyDE Retrieved Document: 0
4155912
How long does it take to remove all wisdom teeth?   I got my wisdom teeth removed 5 days ago. I received intravenous anesthesia, so I was not concious during the process, but those present said it only took about 35 to 40â¦ minutes for removal.
HyDE Retrieved Document: 1
4174313
The time it takes to remove the tooth will vary. Some procedures only take a few minutes, whereas others can take 20 minutes or longer. After your wisdom teeth have been removed, you may experience swelling and discomfort, both on the inside and outside of your mouth.This is usually worse for the first three days, but it can last for up to two weeks. Read more about how a wisdom tooth is removed and recovering from wisdom tooth removal.he time it takes to remove the tooth will vary. Some procedures only take a few minutes, whereas others can take 20 minutes or longer. After your wisdom teeth have been removed, you may experience swelling and discomfort, both on the inside and

### End to End Search

e2e search will directly go through all the steps descripted above.

In [27]:
hits = hyde.e2e_search(query, k=10)
for i, hit in enumerate(hits):
    print(f'HyDE Retrieved Document: {i}')
    print(hit.docid)
    print(json.loads(corpus.doc(hit.docid).raw())['contents'])

HyDE Retrieved Document: 0
4155912
How long does it take to remove all wisdom teeth?   I got my wisdom teeth removed 5 days ago. I received intravenous anesthesia, so I was not concious during the process, but those present said it only took about 35 to 40â¦ minutes for removal.
HyDE Retrieved Document: 1
4174313
The time it takes to remove the tooth will vary. Some procedures only take a few minutes, whereas others can take 20 minutes or longer. After your wisdom teeth have been removed, you may experience swelling and discomfort, both on the inside and outside of your mouth.This is usually worse for the first three days, but it can last for up to two weeks. Read more about how a wisdom tooth is removed and recovering from wisdom tooth removal.he time it takes to remove the tooth will vary. Some procedures only take a few minutes, whereas others can take 20 minutes or longer. After your wisdom teeth have been removed, you may experience swelling and discomfort, both on the inside and